In [1]:
import os
import glob

import numpy as np

from medpy.metric import binary

from PIL import Image

import imageio

In [55]:
# task = '_betnorm' # _gmnorm _ventnorm _wmnorm _betnorm _csfnorm
# gt_dir = '/data/ychan019/experiments/radiaide/radiaide/code/inference/test_kcl_p2p_seg_alt' + task + '/gt/'
# syn_dir = '/data/ychan019/experiments/radiaide/radiaide/code/inference/test_kcl_p2p_seg_alt' + task + '/mask/' # sub-0032_20.png
task = 'bet_alt'
gt_dir = '/data/data_repo/neuro_img/anat_brain_img/datasets/CERMEP-IDB-MRXFDG/processed/yihao/labels_new_test/' + task
syn_dir = '/data/ychan019/experiments/radiaide/Pytorch-UNet-master/data/mask_' + task

In [56]:
all_filepaths_gt = sorted(glob.glob(gt_dir + "/*.*"))
all_filepaths_syn = sorted(glob.glob(syn_dir + "/*.*"))

In [57]:
from collections import defaultdict

prev_subject_id = 'sub-0028'
hist_dc = defaultdict(list)
hist_jc = defaultdict(list)

fp = []
missed = []

for subj_path_gt, subj_path_syn in zip(all_filepaths_gt, all_filepaths_syn):
    
    # gt = np.array(Image.open(subj_path_gt).convert("L"))
    gt = np.array(imageio.imread(subj_path_gt)) # filename
    syn = np.array(Image.open(subj_path_syn).convert("L"))
    
    if np.sum(gt) == 0: # and np.sum(syn) == 0: # if no incl syn this will miss out on false positive
        if np.sum(syn) != 0:
            fp.append((subj_path_gt, subj_path_syn))
        continue
    
    cur_subject_id_full = subj_path_syn.split('/')[-1]
    cur_subject_id = cur_subject_id_full.split('_')[0]
    
    dc = binary.dc(gt, syn) # https://loli.github.io/medpy/generated/medpy.metric.binary.dc.html#medpy.metric.binary.dc
    jc = binary.jc(gt, syn) # https://loli.github.io/medpy/generated/medpy.metric.binary.jc.html#medpy.metric.binary.jc
    print(cur_subject_id_full, dc, jc)
    
    if cur_subject_id != prev_subject_id:
        print()
    
    prev_subject_id = cur_subject_id
    
    if dc != 0 or jc != 0:
        hist_dc[cur_subject_id].append(dc)
        hist_jc[cur_subject_id].append(jc)
    else:
        missed.append((subj_path_gt, subj_path_syn))

sub-0028_100.png 0.9856015307342741 0.9716118079788739
sub-0028_101.png 0.986561000644499 0.9734784247220652
sub-0028_102.png 0.9864117478783255 0.9731878263323769
sub-0028_103.png 0.9859805298360032 0.972348716022722
sub-0028_104.png 0.9848441657164576 0.970140871437263
sub-0028_105.png 0.9843623746375778 0.9692062897791089
sub-0028_106.png 0.9844500451757098 0.9693762876943248
sub-0028_107.png 0.984733187482165 0.969925516466014
sub-0028_108.png 0.9851647567875993 0.9707632479032938
sub-0028_109.png 0.9853681710213776 0.9711583481599401
sub-0028_110.png 0.9844372746251661 0.9693515230797982
sub-0028_111.png 0.9837300177619893 0.9679809843400448
sub-0028_112.png 0.9840329764048138 0.9685678309937975
sub-0028_113.png 0.9846372688477951 0.9697394228075091
sub-0028_114.png 0.9850604469990262 0.9705607039221192
sub-0028_115.png 0.9847566050747901 0.9699709547456198
sub-0028_116.png 0.9845941376765007 0.9696557546196417
sub-0028_117.png 0.9854346945100002 0.9712875939849624
sub-0028_118.pn

In [58]:
all_dc = []

for k,v in hist_dc.items():
    
    all_dc.extend(v)
    
    sub_scores = np.array(v)
    print(k, sub_scores.mean(), sub_scores.std(), sub_scores.min(), sub_scores.max())


sub-0028 0.9374151341671303 0.09468321325436335 0.33864207879295893 0.986561000644499
sub-0029 0.9201420458958156 0.1359064030333475 0.09090909090909091 0.9906322059953881
sub-0030 0.9285220194657772 0.12343328070017048 0.25768667642752563 0.9919736377545015
sub-0031 0.9196598595773106 0.15505211340405223 0.009679370840895343 0.9921748390980228
sub-0032 0.9349212586874868 0.11447075909151744 0.0029850746268656717 0.9901522817582743
sub-0033 0.9297629009196909 0.12662585741422994 0.022988505747126436 0.9887281746518995
sub-0034 0.9455295355706828 0.08419813912639205 0.26251896813353565 0.9955203227243943
sub-0035 0.9367513828366835 0.11708581740728616 0.13793103448275862 0.9928159227417266
sub-0036 0.9234520783781898 0.12247894207955846 0.18292682926829268 0.9856365025006775
sub-0037 0.928065868893146 0.142486788787314 0.027538726333907058 0.989611406721489


In [59]:
all_jc = []

for k,v in hist_jc.items():

    all_jc.extend(v)
    
    sub_scores = np.array(v)
    print(k, sub_scores.mean(), sub_scores.std(), sub_scores.min(), sub_scores.max())


sub-0028 0.8937835383077204 0.13121807506166824 0.20383451059535823 0.9734784247220652
sub-0029 0.8729052357367169 0.16803744579688587 0.047619047619047616 0.9814382942268336
sub-0030 0.8851292662289236 0.16327787767093208 0.14789915966386555 0.9840750945687199
sub-0031 0.8771261046742592 0.18365767626647156 0.004863221884498481 0.9844711936047046
sub-0032 0.8930691434520142 0.14557407601853795 0.0014947683109118087 0.9804966272362889
sub-0033 0.8863591280711987 0.15193018788532472 0.011627906976744186 0.9777076250607091
sub-0034 0.90615273851463 0.12078275739889824 0.15109170305676856 0.9910806014756701
sub-0035 0.8974351842347325 0.15149597825896283 0.07407407407407407 0.985734331150608
sub-0036 0.8756606158956873 0.15894693978646726 0.10067114093959731 0.9716797823763723
sub-0037 0.8881630475131789 0.17119741185628295 0.013961605584642234 0.9794364399051614


In [60]:
all_dc = np.array(all_dc)
all_jc = np.array(all_jc)

In [61]:
all_dc.mean(), all_dc.std()

(0.9303930317950935, 0.12358798548907296)

In [62]:
all_jc.mean(), all_jc.std()

(0.8875463536977474, 0.15603674496901138)

In [63]:
len(fp)

2

In [64]:
len(missed)

45

In [65]:
gt

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [66]:
syn

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [67]:
np.unique(gt)

array([0, 1], dtype=uint8)

In [68]:
np.unique(syn)

array([  0, 127], dtype=uint8)

In [69]:
np.sum(gt), np.sum(syn)/127

(21000, 20613.0)